In [23]:
from __future__ import print_function
import numpy as np
import datetime as dt
from dateutil.parser import isoparse
import pickle
import os.path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [8]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [9]:
# scope
scopes = ['https://www.googleapis.com/auth/calendar']

In [25]:
def get_service(calendar_name='personal'):
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(f'{calendar_name}-token.pickle'):
        with open(f'{calendar_name}-token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                f'{calendar_name}-credentials.json', scopes)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(f'{calendar_name}-token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('calendar', 'v3', credentials=creds)

In [56]:
def get_week_events(service, calendar_id='primary'):
    
    # Setting now time
    now = dt.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    # Getting Monday 5 AM
    today = dt.date.today()
    monday = today - dt.timedelta(days=60)
    week_start = dt.datetime.combine(monday, dt.time(5,0)).isoformat() + 'Z'
    
    return service.events().list(
        calendarId=calendar_id, orderBy='startTime',singleEvents=True, 
        timeMin=week_start, timeMax=now
    ).execute()

In [57]:
# get_service().calendarList().list().execute()

In [58]:
visium_calendar_id = "bjaqb1128v8fgrknb29d8r07is@group.calendar.google.com"

In [128]:
events = pd.DataFrame(data=get_week_events(get_service(), visium_calendar_id).get('items', []))

In [126]:
# Pre-processing
def pre_process(raw):
    # Copy DataFrame
    processed = raw.copy()
    
    # Drop unwanted columns
    for col in processed.columns:
        if col not in ['start', 'end', 'summary', 'colorId', 'location', 'attendees']:
            del processed[col]
            
    processed.drop(processed[processed.summary == "Home office"].index, inplace=True)
            
    # TODO: verify that this modif is okay and doesn't miss out on anything
    idx = pd.isna([date.get('dateTime') for date in processed['end']])
    processed.drop(
        processed[idx].index,
        inplace=True
    )

    # Dates
    processed['end'] = [isoparse(date.get('dateTime')) for date in processed['end']] 
    processed['start'] = [isoparse(date.get('dateTime')) for date in processed['start']]

    processed['duration'] = processed['end'] - processed['start']
    processed['duration'] = [td.seconds / 3600 for td in processed['duration']]


    processed['datetime'] = pd.to_datetime(processed['start'])
    processed['date'] = processed['datetime'].dt.date
    del processed['start']
    del processed['end']

    if 'colorId' in raw.columns:
        # colorId mapping
        processed['colorId'] = raw['colorId'].fillna(0)
        processed['colorId'] = processed['colorId'].astype(int)

        # create activity col
        activity = ['default'] * 12
        activity[4] = 'meditation'
        activity[7] = 'gym'
        activity[11] = 'DAG'

        processed['activity'] = [ activity[colorId] for colorId in processed['colorId'] ]

    if 'attendees' in raw.columns:
        # count attendees
        processed['attendees'] = raw['attendees'].fillna(1)
        processed['attendees'] = [1 if attendees == 1 else len(attendees) for attendees in processed['attendees']]

    return processed

In [129]:
fig = px.bar(pre_process(events), x='date', y='duration')
fig